Import package<br>
取得VD真值

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

VD、CCTV預設路徑

In [2]:
# vd_id = 'nfbVD-N3-S-1.730-M-LOOP'
# vd_id = 'nfbVD-N5-N-16.900-M-PS'
# vd_id = 'nfbVD-N5-S-21.063-M-PS'
# vd_id = 'nfbVD-N5-S-23.910-M-PS'
# vd_id = 'nfbVD-N5-N-25.310-M-PS'
vds = ['nfbVD-N5-S-21.063-M-PS', 'nfbVD-N5-S-23.910-M-PS', 'nfbVD-N5-N-25.310-M-PS']
vds = ['nfbVD-N5-N-16.900-M-PS', 'nfbVD-N5-N-18.313-M-PS', 'nfbVD-N5-N-22.510-M-PS', 'nfbVD-N5-N-25.310-M-PS',
       'nfbVD-N5-S-18.312-M-PS', 'nfbVD-N5-S-19.677-M-PS', 'nfbVD-N5-S-21.063-M-PS', 'nfbVD-N5-S-23.910-M-PS']
vds = ["nfbVD-N5-S-21.063-M-PS", "nfbVD-N5-S-23.910-M-PS", "nfbVD-N5-N-25.310-M-PS"]
cctvs = ['nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M', 'nfbCCTV-N5-N-25.309-M']

起始、結束時間設定(資料區間)

In [3]:
day1 = datetime.strptime('20190728', "%Y%m%d")
dayLast = datetime.strptime('20190729', "%Y%m%d")

產生VD真值 : 會生成三種真值 Spd_Max、Spd_Min、Spd_Avg

In [ ]:
for vd_id in vds:
    vd_df = None
    print(vd_id)
    path = 'D:/vd/' + vd_id
    os.chdir(path)
    day = day1
    while day <= dayLast:
        print("vd_value_" + day.strftime("%Y%m%d"))
        df = pd.read_csv("vd_value_" + day.strftime("%Y%m%d") + ".txt", header=None, sep="\t", index_col=0)
        lanes = 0
        if len(df.columns) == 10:
            print("1 Lane")              #看總共幾道 每多一道 就多9個columns 然後取出Max_speed column
            lanes = 1
            df2 = df[[3]]
        elif len(df.columns) == 19:
            print("2 Lanes")
            lanes = 2
            df2 = df[[3, 12]]
        elif len(df.columns) == 28:
            print("3 Lanes")
            lanes = 3
            df2 = df[[3, 12, 21]]
        else:
            print("4 Lanes")
            lanes = 4
            df2 = df[[3, 12, 21, 30]]
        df2.columns = ['Spd' + repr(x) for x in range(lanes)]
        df2['Spd_Max'] = pd.Series(-1, index=df2.index)
        df2['Spd_Min'] = pd.Series(-1, index=df2.index)
        df2['Spd_Avg'] = pd.Series(-1, index=df2.index)

        df2 = df2[~df2.index.duplicated(keep='first')]   #查看index有沒有重複 有重複 刪除重複的index
        for i in df2.index:
            isDropped = 0
            zero_count = 0
            for x in df2.columns:
                try:
                    if df2.loc[i][x] == -99:
                        df2 = df2.drop(i)
                        isDropped = 1
                        break
                except:
                    print(df2.loc[i])
            if isDropped:
                continue
            df2.loc[i]['Spd_Max'] = max([x for x in df2.loc[i].values[0:lanes]]) #df2.loc[i].values.max()
            try:
                df2.loc[i]['Spd_Min'] = min([x for x in df2.loc[i].values[0:lanes] if x != 0]) #df2.loc[i].values.min()
                df2.loc[i]['Spd_Avg'] = np.mean([x for x in df2.loc[i].values[0:lanes] if x != 0])
            except ValueError:
                df2.loc[i]['Spd_Min'] = 0
                df2.loc[i]['Spd_Avg'] = 0

        vd_df = pd.concat([vd_df, df2])
        day += timedelta(days=1)

    d1str = day1.strftime("%Y%m%d")
    dlaststr = dayLast.strftime("%Y%m%d")
    # vd_df = vd_df.drop(vd_df.columns[0:3], axis=1)
    if not os.path.exists("vd_speed/"):
        os.makedirs("vd_speed/")
    vd_df.to_csv("vd_speed/" + "vdspd_" + d1str + "_" + dlaststr + ".csv")